## <span style=color:blue>Illustrations of python working with SQLite and APSIMX</span>

<span style=color:blue>Some initialization</span>

In [2]:
# These are boiler plate imports that seem useful
# I should probably delete or comment out the ones that aren't used in this script...

import sys
import os
import shutil
# use shutil.rmtree(<path_to_folder>) to do an rm -r on the folder

import json
import csv
import yaml

import pandas as pd
import numpy as np

import matplotlib as mpl
from cycler import cycler

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

# sys.path.append('/Users/rick/HACKING/UTILITIES_IN_HACKING/')
# import utils_general


# this one gives a warning - not fixing for now
from owlready2 import *

# next function is used to generate timestamps that can be used to
# create uniquely named directories and/or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime


print('Initial imports successful')

Initial imports successful


<span style=color:blue>Initializing and using SQLite </span>

In [3]:
# following https://www.chrisnicoll.net/2020/02/exploring-an-sqlite-database-from-jupyter-notebook/

import sqlite3

dir = '/Users/rick/APSIM-v02/WHEAT-ILLUSTRATION/'
sqldb = 'wheat-illustration-v01-FINAL.db'

# bog-standard read-write connection
conn = sqlite3.connect(dir + sqldb)

# create a cursor object
curs = conn.cursor()

# Every SQLite database has an SQLITE_MASTER table that defines the schema for that database

# Use cursor's fetchall() method to fetch contents from sqlite_master
out1 = curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

print(out1)

out2 = curs.execute("SELECT * FROM sqlite_master WHERE type = 'table'").fetchall()
# print(json.dumps(out2, indent=4))

out3 = curs.execute("SELECT * FROM report").fetchall()
print(json.dumps(out3, indent=4))

conn.close()


[('_Factors',), ('_InitialConditions',), ('_Messages',), ('Report',), ('_Simulations',), ('_Checkpoints',), ('_Units',)]
[
    [
        1,
        2,
        "Expt",
        "20",
        "Field",
        "1900-11-10 12:00:00",
        90.0,
        "HarvestRipe",
        739.8777063237192,
        5.370057247880293,
        1969.7904486904256,
        10.081061170973367,
        0.01754423752022418,
        11227.563730938678,
        196.97904486904255,
        3.4776844145792043,
        870.4237032772908
    ],
    [
        1,
        2,
        "Expt",
        "20",
        "Field",
        "1901-11-11 12:00:00",
        90.0,
        "HarvestRipe",
        428.9342454068576,
        2.974703190114613,
        1233.4322380003337,
        9.35184337187275,
        0.016361650290744507,
        7538.55641749087,
        123.34322380003337,
        2.020116479789421,
        499.94244308257294
    ],
    [
        1,
        2,
        "Expt",
        "20",
        "Field",
       

### <span style=color:blue>Leveraging pandas dataframes with sqlite </span>

<span style=color:blue>Note: we should remember to close connections to sqlite databases... </span>

In [4]:
# following https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html

import pandas as pd
import sqlite3

dir = '/Users/rick/APSIM-v02/WHEAT-ILLUSTRATION/'
sqldb = 'wheat-illustration-v01-FINAL.db'

# bog-standard read-write connection
conn = sqlite3.connect(dir + sqldb)

# create a cursor object
curs = conn.cursor()


df1 = pd.read_sql_query("SELECT * from sqlite_master WHERE type = 'table'", conn)
print('\nContents of sqlite_master of type table: \n\n')
print(df1.head(30))


df2 = pd.read_sql_query("SELECT * from report", conn)
print('\n\nSome records from the table "report": \n\n')
print(df2.head(30))

conn.close()


Contents of sqlite_master of type table: 


    type                name            tbl_name  rootpage   
0  table            _Factors            _Factors         2  \
1  table  _InitialConditions  _InitialConditions         4   
2  table           _Messages           _Messages        20   
3  table              Report              Report       674   
4  table        _Simulations        _Simulations       994   
5  table        _Checkpoints        _Checkpoints       996   
6  table              _Units              _Units       998   

                                                 sql  
0  CREATE TABLE [_Factors] ("CheckpointID" intege...  
1  CREATE TABLE [_InitialConditions] ("Checkpoint...  
2  CREATE TABLE [_Messages] ("CheckpointID" integ...  
3  CREATE TABLE [Report] ("CheckpointID" integer,...  
4  CREATE TABLE [_Simulations] ("ID" integer,"Nam...  
5  CREATE TABLE [_Checkpoints] ("ID" integer,"Nam...  
6  CREATE TABLE [_Units] ("TableName" text,"Colum...  


Some records fro

### <span style=color:blue>Invoking APSIM-NextGen (aka APSIMx) on an .apsimx file </span>

<span style=color:blue>We are running a terminal command, using the subprocess package</span>

In [5]:
import subprocess
import os

# note: some of the more recent versions of APSIMX don't work on mac, e.g., the following command
# apsim_cmd = "/Applications/APSIM2023.3.7170.0.app/Contents/Resources/bin/Models"
apsim_cmd = "/Applications/APSIM2022.12.7130.0.app/Contents/Resources/bin/Models"

dir_to_run_in = "/Users/rick/APSIM-v02/WHEAT-ILLUSTRATION/"
model_name = "wheat-illustration-v01-FINAL"

os.chdir(dir_to_run_in)
result1 = subprocess.run(["pwd"], capture_output=True, text=True)
print("result1: \n")
print(result1.stdout)

result2 = subprocess.run([apsim_cmd, model_name + ".apsimx"], capture_output=True, text=True)
print("result2: \n")
print(result2.stdout)

result3 = subprocess.run(["ls -l " + model_name + "*"], shell=True, capture_output=True, text=True)
print("result3: \n ")
print(result3.stdout)



result1: 

/Users/rick/APSIM-v02/WHEAT-ILLUSTRATION

result2: 


result3: 
 
-rw-------@ 1 rick  staff    37462 Apr 24 15:36 wheat-illustration-v01-FINAL.apsimx
-rw-------@ 1 rick  staff    37462 Apr 24 15:35 wheat-illustration-v01-FINAL.apsimx.bak
-rw-r--r--@ 1 rick  staff  4104192 Apr 25 16:27 wheat-illustration-v01-FINAL.db
-rw-r--r--@ 1 rick  staff    32768 Apr 25 04:09 wheat-illustration-v01-FINAL.db-shm
-rw-r--r--@ 1 rick  staff  1685112 Apr 25 16:27 wheat-illustration-v01-FINAL.db-wal



### <span style=color:blue>Defining a couple of functions, so that we can run multiple simulations from python</span>

### <span style=color:blue>Also, adding capability to catch errors messages</span>

In [9]:
def list_model_files(dir, model_name):
    os.chdir(dir)
    arg = 'ls -l ' + model_name + '*'
    result = subprocess.run([arg], shell=True, capture_output=True, text=True)
    return 'The files in ' + dir + ' with prefix "' + model_name + '" are: \n\n' + result.stdout + '\n'

def run_apsimx(dir, model_name):
    apsimx_cmd = "/Applications/APSIM2022.12.7130.0.app/Contents/Resources/bin/Models"
    # note: some of the more recent versions of APSIMX don't work on mac
    # apsim_cmd = "/Applications/APSIM2023.3.7170.0.app/Contents/Resources/bin/Models"
    
    os.chdir(dir)
    try:
        # subprocess.check_output([apsimx_cmd, model_name + '.apsimx'], capture_output=True, text=True) 
        subprocess.check_output([apsimx_cmd, model_name + '.apsimx'], text=True) 
        out = ''
    except subprocess.CalledProcessError as e:
        # following https://stackoverflow.com/questions/24849998/how-to-catch-exception-output-from-python-subprocess-check-output
        out = '\nAn error was found with ' + model_name + '.apsimx, which is as follows: \n'
        out += e.output
        out += '\nEnd of error report\n----------------------------------------------\n'
    return(out)
    

### <span style=color:blue>Invoking APSIMX on a model where we modified the soil component.  We created the model using the Rmd script at https://rpubs.com/rhull_at_acm_dot_org/1033304 </span>

In [15]:


dir = '/Users/rick/AG-CODE--R/R--Fetch-Soil-Profiles--v01/MODELS/'
model_name = 'Maize-lonlat_m75.5_41.5'

out = run_apsimx(dir, model_name)
if out != '':
    print(out)



An error was found with Maize-lonlat_m75.5_41.5.apsimx, which is as follows: 
----------------------------------------------
ERROR in file: /Users/rick/AG-CODE--R/R--Fetch-Soil-Profiles--v01/MODELS/Maize-lonlat_m75.5_41.5.apsimx
Simulation name: Simulation
Models.Core.SimulationException
 ---> System.Exception: Saturation of 0.388 in layer 5 is above acceptable value of 0.358. You must adjust bulk density to below 1.622 OR saturation to below 0.358
Saturation of 0.386 in layer 6 is above acceptable value of 0.343. You must adjust bulk density to below 1.628 OR saturation to below 0.343

   at Models.Soils.Soil.Check(ISummary summary) in /tmp/ApsimX/Models/Soils/Soil.cs:line 172
   at Models.Soils.Soil.OnDoInitialSummary(Object sender, EventArgs e) in /tmp/ApsimX/Models/Soils/Soil.cs:line 125
   at Models.Clock.OnDoCommence(Object sender, CommenceArgs e) in /tmp/ApsimX/Models/Clock.cs:line 304
   at Models.Core.Simulation.Run(CancellationTokenSource cancelToken) in /tmp/ApsimX/Models/C